In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import glob
df = {}
for file in glob.glob('*.csv'):
    name = file.split('.')[0]
    df[name] = pd.read_csv(file)
    print('{0} with shape {1}'.format(name, df[name].shape))

rating with shape (20000263, 4)
tag with shape (465564, 4)
genome_scores with shape (11709768, 3)
genome_tags with shape (1128, 2)
link with shape (27278, 3)
movie with shape (27278, 3)


In [3]:
df['genome_scores'].head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [4]:
movie_tag_scores = df['genome_scores'].merge(df['genome_tags'], on='tagId')

In [5]:
user_tag_scores = df['tag'].merge(movie_tag_scores, how='left', on=['movieId', 'tag']).head()
user_tag_scores.head()

,userId,movieId,tag,timestamp,tagId,relevance
0,18,4141,Mark Waters,2009-04-24 18:19:40,NaN,NaN
1,65,208,dark hero,2013-05-10 01:41:18,288.0,0.94950
2,65,353,dark hero,2013-05-10 01:41:19,288.0,0.99800
3,65,521,noir thriller,2013-05-10 01:39:43,712.0,0.98900
4,65,592,dark hero,2013-05-10 01:41:18,288.0,0.99775


In [6]:
user_ratings = df['rating']
user_ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [7]:
genres = list(set('|'.join(df['movie'].genres).split('|')))
genres = [x if x!='(no genres listed)' else 'NA' for x in genres]
genres

['Adventure',
 'Horror',
 'Film-Noir',
 'Action',
 'Comedy',
 'Western',
 'Fantasy',
 'IMAX',
 'Drama',
 'Musical',
 'Documentary',
 'Animation',
 'NA',
 'Mystery',
 'Romance',
 'War',
 'Sci-Fi',
 'Children',
 'Crime',
 'Thriller']

In [8]:
import re
movies = df['movie'].replace({'(no genres listed)':'NA'})
for genre in genres:
    movies[genre] = 0
    movies.loc[movies.genres.str.contains(genre), genre] = 1
movies['year'] = movies.title.apply(lambda x: re.findall('\((\d{4})\)', x)[0] if len(re.findall('\((\d{4})\)', x))>0 else '0')
movies.title = movies.title.apply(lambda x: x[:-7])
movies = movies.drop(['genres'], axis=1)
movies.head()

,movieId,title,Adventure,Horror,Film-Noir,Action,Comedy,Western,Fantasy,IMAX,...,Animation,NA,Mystery,Romance,War,Sci-Fi,Children,Crime,Thriller,year
0,1,Toy Story,1,0,0,0,1,0,1,0,...,1,0,0,0,0,0,1,0,0,1995
1,2,Jumanji,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1995
2,3,Grumpier Old Men,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1995
3,4,Waiting to Exhale,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1995
4,5,Father of the Bride Part II,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995


In [9]:
movies.iloc[:,2:-1].sum()

Adventure       2329
Horror          2611
Film-Noir        330
Action          3520
Comedy          8374
Western          676
Fantasy         1412
IMAX             196
Drama          13344
Musical         1036
Documentary     2471
Animation       1027
NA               246
Mystery         1514
Romance         4127
War             1194
Sci-Fi          1743
Children        1139
Crime           2939
Thriller        4178
dtype: int64

In [10]:
movies.year.value_counts().sort_index()

0         22
1891       1
1893       1
1894       2
1895       2
        ... 
2011    1016
2012    1022
2013    1011
2014     740
2015     120
Name: year, Length: 119, dtype: int64

In [20]:
user_ratings.groupby('movieId').rating.agg(['mean','count','sum']).sort_values(by=['sum']).merge(movies, on='movieId')

,movieId,mean,count,sum,title,Adventure,Horror,Film-Noir,Action,Comedy,...,Animation,NA,Mystery,Romance,War,Sci-Fi,Children,Crime,Thriller,year
0,105390,0.500000,1,0.5,"Rabid Grannies (Mémés cannibales, Les)",0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1988
1,105481,0.500000,1,0.5,Blood Shack,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1971
2,121463,0.500000,1,0.5,The End,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,2012
3,121467,0.500000,1,0.5,Shark Attack,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1999
4,129885,0.500000,1,0.5,Le ultime ore di una vergine,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26739,260,4.190672,54502,228400.0,Star Wars: Episode IV - A New Hope,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1977
26740,593,4.177057,63299,264403.5,"Silence of the Lambs, The",0,1,0,0,0,...,0,0,0,0,0,0,0,1,1,1991
26741,356,4.029000,66172,266607.0,Forrest Gump,0,0,0,0,1,...,0,0,0,1,1,0,0,0,0,1994
26742,296,4.174231,67310,280967.5,Pulp Fiction,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,1994


In [30]:
# user_ratings.pivot(index='movieId', columns='userId', values='rating').head()